In [58]:
from lz4 import block
import sys
import ast
import pandas as pd
import polars as pl
from flatten_json import flatten
import re

In [2]:
# decompresse lz4 to byte stream
def lz4_to_binary(filepath):
    bytestream = open(filepath, "rb")
    bytestream.read(8)  # skip past the b"mozLz40\0" header
    valid_bytes = bytestream.read()
    binary = block.decompress(valid_bytes)
    return binary

In [31]:
file = './bookmarks.jsonlz4'
binary = lz4_to_binary(file)
json = ast.literal_eval(binary.decode("utf-8"))

In [32]:
type(json)

dict

In [65]:
data = pd.json_normalize(
    json, 
    record_path=['children']
)
# , 
#     meta=['class', 'room', ['info', 'teachers', 'math']],
#     errors='ignore'

In [66]:
data

,guid,title,index,dateAdded,lastModified,id,typeCode,type,root,children
0,menu________,menu,0,1530066880767000,1603076605369000,2,2,text/x-moz-place-container,bookmarksMenuFolder,"[{'guid': 'u32Lc0awE474', 'title': 'Recent Tag..."
1,toolbar_____,toolbar,1,1530066880767000,1648246889622000,3,2,text/x-moz-place-container,toolbarFolder,"[{'guid': 'f_vRq0TY8Fc1', 'title': 'Most Visit..."
2,unfiled_____,unfiled,3,1530066880767000,1648245293660000,5,2,text/x-moz-place-container,unfiledBookmarksFolder,"[{'guid': 'y_PUzyeBeUrp', 'title': 'boilerplat..."
3,mobile______,mobile,4,1530066880776000,1584131412530000,6,2,text/x-moz-place-container,mobileFolder,NaN


In [37]:
flat = flatten(json, separator='.')

In [60]:
r = re.compile("^.*\.uri$")
uris = list(filter(r.match, list(flat.keys())))
len(uris)

11527

In [69]:
r = re.compile("^.*\.id$")
ids = list(filter(r.match, list(flat.keys())))
len(ids)

24422

In [112]:
r = re.compile('^.*\.{1}[a-z|A-Z]*$')
endings = list(filter(r.match, list(flat.keys())))
len(endings)

212633

In [114]:
ends = [re.search('\.{1}[a-z|A-Z]*$', k)[0] for k in endings]
len(ends)

212633

In [115]:
set(ends)

{'.charset',
 '.dateAdded',
 '.guid',
 '.iconuri',
 '.id',
 '.index',
 '.lastModified',
 '.root',
 '.title',
 '.type',
 '.typeCode',
 '.uri'}

In [116]:
# ^ for each uri - grab a record for all titles in hiearchy
# need logic to go up or down the tree ...

In [107]:
re.search('\.{1}[a-z]*$', list(flat.keys())[9])[0]

'.guid'

In [68]:
# 0 is menu    - recent tags & work latest
# 1 is toolbar - all the bookmarks
# 2 is unfiled - todo & boilerplate
# 3 is mobile  - nothing ... yet (: 
data.loc[:,'children'][0]

[{'guid': 'u32Lc0awE474',
  'title': 'Recent Tags',
  'index': 0,
  'dateAdded': 1530066882659000,
  'lastModified': 1595453963000000,
  'id': 12,
  'typeCode': 1,
  'type': 'text/x-moz-place',
  'uri': 'place:type=6&sort=14&maxResults=10'},
 {'guid': '6aOuabmYunkb',
  'title': 'Work Latest',
  'index': 1,
  'dateAdded': 1598660067092000,
  'lastModified': 1600455199343000,
  'id': 17498,
  'typeCode': 2,
  'type': 'text/x-moz-place-container'}]

In [15]:
pd.json_normalize(json)

,guid,title,index,dateAdded,lastModified,id,typeCode,type,root,children
0,root________,,0,1530066880767000,1648246889622000,1,2,text/x-moz-place-container,placesRoot,"[{'guid': 'menu________', 'title': 'menu', 'in..."


In [ ]:
# does not work obviously
d_flat = pd.DataFrame(flat)